# Deep Gaussian Process (DGP)

In [ ]:

#@title install packages
!pip install gpytorch
!pip install optuna
!pip install watermark

In [ ]:
#@title import packages
import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import torch
from torch.utils.data import TensorDataset, DataLoader

import gpytorch
from torch.nn import Linear
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL

sns.reset_defaults()
sns.set_context(context='talk', font_scale=1.0)
cmap = plt.get_cmap("tab10")

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:

def set_seed(seed):
    # random
    # random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [ ]:

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv')
df.info()
df.head()

In [ ]:

X, y = df.iloc[:, 1:], df.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 説明変数、観測変数の標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 参考文献では元のスケールにおけるRMSEを算出しているため、統計量を求めておく。
m, s = y_train.mean(), y_train.std(ddof=0) #ddof : Delta degree of freedom, ddof=0: entire data is included->divided by n, ddof=1:samples from entire data->divided by (n-1)
y_train = (y_train.values - m) / s
y_test = (y_test.values - m) / s

dtype = torch.float32
X_train, X_test, y_train, y_test = (
    torch.tensor(X_train, dtype=dtype),
    torch.tensor(X_test, dtype=dtype),
    torch.tensor(y_train, dtype=dtype),
    torch.tensor(y_test, dtype=dtype)
    )

# データをGPUに配置
if torch.cuda.is_available():
    (X_train, X_test, y_train, y_test) = (
        X_train.cuda(), X_test.cuda(), y_train.cuda(), y_test.cuda()
    )

# ミニバッチを読み込むためのDataLoaderを作成
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

## Model definition

In [ ]:
class DeepGPHiddenLayer(DeepGPLayer):
  def __init__(self,input_dims,output_dims,num_inducing=128,mean_type="constant"):
    if output_dims is None:
      #set initial points
      inducing_points  = torch.randn(num_inducing,input_dims) #(num_inducing,input_dims)
      batch_shape=torch.Size([])
    else:
      inducing_points = torch.randn(output_dims,num_inducing,input_dims)
      batch_shape=torch.Size([output_dims])

    #set approximate posterior distribution
    variational_distribution = CholeskyVariationalDistribution(
        num_inducing_points=num_inducing,
        batch_sahpe=batch_shape
    )

    variational_strategy = VariationalStrategy(
        self,
        inducing_points,
        variational_distribution,
        learn_inducing_locations=True #position of inducing points is set trainable
    )

    super(DeepGPHiddenLayer,self).__init__(variational_strategy,input_dims,output_dims)

    #mean function
    if mean_type=="constant":
      self.mean_module=ConstantMean(batch_shape=batch_shape)
    else:
      self.mean_module=LinearMean(input_dims)
    self.covar_module = ScaleKernel(
        #RBF kernel
        RBFKernel(batch_shape=batch_shape,ard_num_dims=input_dims),
        #"""ARD : Automatic Relevance Determination
        #ARD allows different input dimensions/features to have different length scales or relevance. ard_num_dims typically specifies the number of input dimensions for which ARD will be applied, and input_dims likely provides this number.
        #"""
        batch_shape=batch_shape,
        ard_num_dims=None
    )

  def forward(self,x):
    mean_x = self.mean_module(x)
    covar_x = self.covar_module(x)
    return MultivariateNormal(mean_x,covar_x)


In [ ]:
class DGP(DeepGP):
  def __init__(self,input_dim,hidden_dim):
    hidden_layer = DeepGPHiddenLayer(
        input_dims=input_dim,
        output_dims=hidden_dim,
        #use linear mean function
        mean_type="linear",
    )

    last_layer = DeepGPHiddenLayer(
        input_dims=hidden_layer.output_dims,
        output_dims=None,
        mean_type="constant",
    )

    super().__init__()

    self.hidden_layer = hidden_layer
    self.last_layer = last_layer
    self.likelihood = GaussianLikelihood()

  def forward(self,inputs):
    hidden_rep1 = self.hidden_layer(inputs)
    output =  self.last_layer(hidden_rep1)
    return output

  def predict(self,test_loader):
    with torch.no_grad(): #suppress gradient calculation
      mus = [] #mu
      variances = [] #covar
      lls = [] #log likelihood
      for x_batch,y_batch in test_loader:
        preds = self.likelihood(self(x_batch))
        mus.append(preds.mean)
        variances.append(preds.variance)
        lls.append(self.likelihood.log_marginal(y_batch,self(x_batch)))

    return torch.cat(mus,dim=-1), torch.cat(variances,dim=-1),torch.cat(lls,dim=-1)

input_dim = hidden_dim = X_train.shape[-1]
model = DGP(input_dim,hidden_dim)
if torch.cuda.is_available():
  model=model.cuda()

## Inference with Variatinal inference
- approximate ELBO with sampling

In [ ]:
num_epochs = 200
num_samples = 10

optimizer = torch.optim.Adam([{"params":model.parameters()}],lr=0.01)
#objective fn
mll = DeepApproximateMLL(VariationalELBO(model.likelihood,model,X_train.shape[-2]))

losses = []
epochs_iter = tqdm.notebook.tqdm(range(num_epochs),desc="Epoch")
for i in epochs_iter:
  epoch_loss = []
  for x_batch,y_batch in train_loader:
    with gpytorch.settings.num_likelihood_samples(num_samples):
      optimizer.zero_grad()
      output=model(x_batch)
      loss=-mll(output,y_batch)
      loss.backward()
      optimizer.step()
      epoch_loss.append(loss.item())
  losses.append(np.mean(epoch_loss))

## predict

In [ ]:
model.eval()
#mean,varicance,log likelihood
predictive_means,predictive_variances,test_lls = model.predict(test_loader)

mse = mean_squared_error(y_test.cpu(),predictive_means.mean(0).cpu())*s**2

print(f"RMSE(DGP):{mse**0.5:.2f}")
print(f"Log Likelihood(DGP) :{test_lls.mean().item():.2f} ")

In [ ]:
test_lls